In [4]:
# 查看模型
from models.Stgcn import *
import networkx as nx
import numpy as np
from networkx import adjacency_matrix
from networkx.generators import grid_2d_graph, empty_graph
import matplotlib.pyplot as plt

adj_mx = adjacency_matrix(grid_2d_graph(20, 20))
L_tilde = scaled_Laplacian(adj_mx)

all_backbones = [
    {'K': 3, 'nb_chev_filter': 64, 'nb_time_filter': 64, 'in_len': 6, 'time_strides': 1},  # recent
    # {'K': 3, 'nb_chev_filter': 64, 'nb_time_filter': 64, 'in_len': p['period_len'], 'time_strides': 1},  # daily
    # {'K': 3, 'nb_chev_filter': 64, 'nb_time_filter': 64, 'in_len': p['trend_len'], 'time_strides': 1}, # weekend
]

astgcn = ASTGCN(all_backbones, adj_mx, in_len=6, pred_len=6)
# dummy_input = torch.rand(32, 400, 1, 6) # [batch_size, N, F_in, T]
# out = astgcn(dummy_input)

print(astgcn)

ASTGCN(
  (criterion): L1Loss()
  (valid_MAE): MeanAbsoluteError()
  (valid_MAPE): MeanAbsolutePercentageError()
  (valid_SMAPE): SymmetricMeanAbsolutePercentageError()
  (valid_R2): R2Score()
  (valid_RMSE): MeanSquaredError()
  (test_MAE): MeanAbsoluteError()
  (test_MAPE): MeanAbsolutePercentageError()
  (test_SMAPE): SymmetricMeanAbsolutePercentageError()
  (test_R2): R2Score()
  (test_MSE): MeanSquaredError()
  (submodules): ModuleList(
    (0): ASTGCN_submodule(
      (BlockList): ModuleList(
        (0): ASTGCN_block(
          (TAt): Temporal_Attention_layer()
          (SAt): Spatial_Attention_layer()
          (cheb_conv_SAt): cheb_conv_withSAt(
            (Theta): ParameterList(
                (0): Parameter containing: [torch.float32 of size 1x64]
                (1): Parameter containing: [torch.float32 of size 1x64]
                (2): Parameter containing: [torch.float32 of size 1x64]
            )
          )
          (time_conv): Conv2d(64, 64, kernel_size=(1, 3), 

#### 真正有效的模块

Input 0 shape=torch.Size([32, 400, 1, 128])

Training started: y.shape=torch.Size([32, 32, 1, 20, 20])

(b, N, F_in, T) # 对(N, F_in)维度做图滤波，通过相邻节点特征聚合，将节点特征F维度由1扩展为64

(ChebConv): ChebConv((Theta))

=> (b, N, F_out, T) # 对(N,T)维度做卷积，核(1,3)，因此实际上仅对T维度做了卷积

(time_conv): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))

=> (b, N, F_out, T) # 残差连接，为匹配维度加了个卷积，把节点特征维度特扩展到了64。对性能影响不大。

(residual_conv): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))

=> (b, N, F_out, T) # 对F_out维度做层归一化

(ln): LayerNorm((64,), eps=1e-05, elementwise_affine=True)

=> (b, T, N, F_in) # 对(N,F_in)做卷积，核(1,64)，实际上仅对F维度做卷积，用64的核把F_out(64)恢复为F_in(1)。时间维度作为独立通道。

(final_conv): Conv2d(6, 6, kernel_size=(1, 64), stride=(1, 1))





In [ ]:
# 查看图 G
from models.Stgcn import *
import networkx as nx
import numpy as np
from networkx import adjacency_matrix
from networkx.generators import grid_2d_graph, empty_graph
import matplotlib.pyplot as plt


# 创建一个包含400个节点但没有任何边的图
# G = nx.empty_graph(20*20)
G = nx.grid_2d_graph(20, 20) # 用的是网格图，k_average = 3.8
# G = nx.erdos_renyi_graph(400, 3.8/400, seed=42) # k_average = N*p
# G = nx.barabasi_albert_graph(400, 2, seed=42) # k_average = 2N
# 为每个节点添加自环
# for node in G.nodes():
#     G.add_edge(node, node)

# adj_mx = np.load("/data/scratch/jiayin/AdjKnn5_Milan10Min_Internet.npy")
adj_mx = nx.adjacency_matrix(G)
L_tilde = scaled_Laplacian(adj_mx)
# L_tilde = np.diag([1] * 25)

N = adj_mx.shape[0]

# 计算平均度
average_degree = np.mean(np.sum(adj_mx, axis=1))
print(f"节点平均度: {average_degree}")

# 绘制邻接矩阵的图
# plt.figure(figsize=(6, 6))
# # plt.imshow(adj_mx, cmap='viridis', interpolation='none')
# plt.imshow(adj_mx.todense().astype(np.float32), cmap='viridis', interpolation='none')
# plt.title('erdos_renyi_graph(400, p = 3.8/400)')
# plt.colorbar()
# plt.xlabel('Node Index')
# plt.ylabel('Node Index')
# plt.show()

# 绘制每个 Chebyshev 多项式矩阵的图
# K = 2
# cheb_polynomials = [np.identity(N), np.asarray(L_tilde)]
# for i in range(2, K):
#     print("i = ", i)
#     cheb_polynomials.append(np.asarray(2 * L_tilde * cheb_polynomials[i - 1] - cheb_polynomials[i - 2]))
# cheb_polynomials = np.stack(cheb_polynomials, axis=0).astype(np.float32)
# for i, cheb_poly in enumerate(cheb_polynomials):
#     plt.figure(figsize=(4, 4))
#     plt.imshow(cheb_poly, cmap='viridis', interpolation='none')
#     plt.title(f'Chebyshev Polynomial Matrix T_{i}')
#     plt.colorbar()
#     plt.show()

# 计算 L_tilde 的特征值
# eigenvalues = np.linalg.eigvals(L_tilde)
#
# # 获取最大和最小特征值
# max_eigenvalue = np.max(eigenvalues)
# min_eigenvalue = np.min(eigenvalues)
#
# print(f"Max eigenvalue: {max_eigenvalue}")
# print(f"Min eigenvalue: {min_eigenvalue}")

# 创建 20x20 的 2D 网格图
# G = grid_2d_graph(5, 5)

# 使用 networkx 绘制图
# plt.figure(figsize=(4, 4))
# pos = {node: (node[1], -node[0]) for node in G.nodes()}  # 为节点设置位置
# nx.draw(G, pos, with_labels=True, node_size=50, font_size=8, font_weight='bold', edge_color='gray')
#
# plt.title('2D Grid Graph Topology')
# plt.show()


In [1]:
# 查看维度
from models.Stgcn import *
import networkx as nx
import numpy as np
torch.manual_seed(42)

G = nx.grid_2d_graph(20, 20) # 用的是网格图，k_average = 3.8
adj_mx = nx.adjacency_matrix(G)
L_norm = scaled_Laplacian(adj_mx)
spatial_At = torch.ones(32, 400, 400)

chevGF = cheb_conv_withSAt(K=3, in_channels=1, out_channels=64, L_tilde = L_norm)
dummy_input = torch.rand(32,400,1,128) # b,N,F_in,T
out = chevGF(dummy_input, spatial_At) # b,N,F_out,T

print(chevGF)

# 循环：5s
# Mean of out: inf
# Standard Deviation of out: inf
# First 10 elements of out: tensor([0.0000e+00, 0.0000e+00, 1.6681e-26, 0.0000e+00, 0.0000e+00, 4.1931e-27,
#         5.8752e-27, 0.0000e+00, 0.0000e+00, 4.6099e-27],
#        grad_fn=<SliceBackward0>)



cheb_conv_withSAt(
  (Theta): ParameterList(
      (0): Parameter containing: [torch.float32 of size 1x64]
      (1): Parameter containing: [torch.float32 of size 1x64]
      (2): Parameter containing: [torch.float32 of size 1x64]
  )
)


In [ ]:
# 试图用 tensorboard 可视化模型结构/维度，没成功
from models.Stgcn import *
import networkx as nx
import torch
from networkx import adjacency_matrix
from networkx.generators import grid_2d_graph
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from tensorboard import notebook

%load_ext tensorboard

# 创建 SummaryWriter 实例
writer = SummaryWriter(log_dir='./my_tensorboard_logs')

adj_mx = adjacency_matrix(grid_2d_graph(20, 20))

L_tilde = scaled_Laplacian(adj_mx)

astgcn_block = ASTGCN_block(L_tilde, in_channels=1, K=3, nb_chev_filter=64, nb_time_filter=64, time_strides=1, num_of_vertices=400, num_of_timesteps=5)
dummy_input = torch.rand(32, 400, 1, 5) # [batch_size, N, F_in, T]
out = astgcn_block(dummy_input)

%tensorboard --logdir=./my_tensorboard_logs --port=6008

# 将模型结构写入 TensorBoard
writer.add_graph(astgcn_block, dummy_input)
notebook.list() # View open TensorBoard instances
notebook.display(port=6008)

# 关闭 writer
writer.close()